# BioBatchNet Tutorial

**BioBatchNet** is a dual-encoder VAE framework for batch effect correction, supporting both:
- **Imaging Mass Cytometry (IMC)** data
- **Single-cell RNA-seq (scRNA-seq)** data

This tutorial covers:
1. **Quick Start** — default parameters
2. **Custom Loss Weights** — fine-tuning training objectives

## 1. Installation and Setup

In [ ]:
# Install BioBatchNet (uncomment if needed)
# !pip install biobatchnet[tutorial]

import torch
import scanpy as sc
from biobatchnet import correct_batch_effects

print(f"PyTorch: {torch.__version__}")
print(f"Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

## 2. Load Example Data

We'll use the **IMMUcan IMC dataset** as an example. The data will be downloaded from Google Drive.

In [ ]:
import os
import gdown

FILE_ID = "1S0AgcT0J7tnRtnnshRzAkECwhse0mTrK"
FILENAME = "IMMUcan_batch.h5ad"

if not os.path.exists(FILENAME):
    gdown.download(id=FILE_ID, output=FILENAME, quiet=False)

adata = sc.read_h5ad(FILENAME)
print(f"Loaded: {adata.n_obs:,} cells, {adata.n_vars} features, "
      f"{adata.obs['BATCH'].nunique()} batches")

In [ ]:
# Visualize original data (before correction)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.pl.umap(adata, color=['BATCH', 'celltype'], frameon=False, wspace=0.5,
           title=['Original — Batch', 'Original — Cell type'])

---

## 3. Quick Start

Just pass your AnnData object — BioBatchNet handles the rest.

In [ ]:
bio_emb, batch_emb = correct_batch_effects(
    adata,
    batch_key='BATCH',
    data_type='imc',
    epochs=100,
)

adata.obsm['X_biobatchnet'] = bio_emb
print(f"Bio embeddings: {bio_emb.shape}")

# Visualize corrected data
sc.pp.neighbors(adata, use_rep='X_biobatchnet')
sc.tl.umap(adata)
sc.pl.umap(adata, color=['BATCH', 'celltype'], frameon=False, wspace=0.5,
           title=['Corrected — Batch', 'Corrected — Cell type'])

---

## 4. Custom Loss Weights

Override specific loss weights to control batch mixing strength.

In [ ]:
bio_emb_custom, _ = correct_batch_effects(
    adata,
    batch_key='BATCH',
    data_type='imc',
    epochs=100,
    loss_weights={
        'discriminator': 0.1,  # lower = stronger batch mixing (default: 0.3)
        'kl_bio': 0.01,        # bio encoder regularization (default: 0.005)
    },
)

adata.obsm['X_custom'] = bio_emb_custom

sc.pp.neighbors(adata, use_rep='X_custom')
sc.tl.umap(adata)
sc.pl.umap(adata, color=['BATCH', 'celltype'], frameon=False, wspace=0.5,
           title=['Custom weights — Batch', 'Custom weights — Cell type'])

---

## 5. Loss Weight Reference

| Parameter | IMC default | scRNA-seq default | Description |
|-----------|-------------|-------------------|-------------|
| `recon` | 10.0 | 10.0 | Reconstruction quality |
| `discriminator` | 0.3 | 0.04 | Batch mixing (lower = more mixing) |
| `classifier` | 1.0 | 1.0 | Batch info retention |
| `kl_bio` | 0.005 | 1e-6 | Bio encoder regularization |
| `kl_batch` | 0.1 | 0.01 | Batch encoder regularization |
| `ortho` | 0.01 | 0.0002 | Bio/batch orthogonality |
| `kl_size` | — | 0.002 | Size factor (seq only) |

For more details, see [README.md](README.md).